#프로젝트의 결론
1. 가진 데이터에 비해 네트워크가 너무 깊으면 overfitting한다.
2. 네트워크가 너무 얕아도 underfitting 문제가 일어남.
3. learning rate가 학습 성공에 큰 영향을 끼친다.
 * 0.03때는 제대로 학습이 안되고 default인 0.001로 하니까 잘됐다.

Conv2d 사용했을때보다 Linear만 사용했을때 더 성능 잘나옴.
Conv2d 파라미터인 padding, stride 같은 값들을 제대로 설정못해줘서 그런것같다고 생각함.


In [0]:
import torch
import torchvision 

In [2]:
trans = torchvision.transforms.Compose([
                                              torchvision.transforms.ToTensor()
])

#data 가져오기
train_dataset = torchvision.datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, transform=trans)
val_dataset = torchvision.datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, transform=trans)
test_dataset = torchvision.datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=False, transform=trans)

indices = list(range(len(train_dataset.train_data)))
split = int(len(indices)*0.2)
train_idx, val_idx = indices[:split], indices[split:]

train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_idx)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=32, sampler=train_sampler)
val_sampler = torch.utils.data.sampler.SubsetRandomSampler(val_idx)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=32, sampler=val_sampler)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=32)


0it [00:00, ?it/s]

26427392it [00:01, 14843406.17it/s]                             


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 100152.41it/s]                           
0it [00:00, ?it/s]

...

4423680it [00:01, 4310918.69it/s]                             
0it [00:00, ?it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw


8192it [00:00, 25526.93it/s]            


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:53: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [0]:
from torch import nn

Fashion data는 28*28 grayscale 이미지를 사용하기 때문에
Convolutional layer를 사용하는 것이 더 좋은 성능을 낼 것이라고 생각함.

모델 구성

1. conv
2. maxpool
3. conv
4. maxpool
5. conv
6. fc
7. fc


In [0]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # self.conv1 = nn.Conv2d(1, 128, 3)
        # self.conv2 = nn.Conv2d(128, 256, 3, stride=2)
        # self.conv3 = nn.Conv2d(256, 512, 3, stride=2)
        # self.conv4 = nn.Conv2d(512, 64, 3)
        # self.maxpool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(28*28,128)
        self.fc2 = nn.Linear(128,256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64,10)
        self.drop = nn.Dropout()
        

    def forward(self, x):
        # x = self.maxpool(nn.functional.relu(self.conv1(x)))
        # x = self.maxpool(nn.functional.relu(self.conv2(x)))
        # x = self.maxpool(nn.functional.relu(self.conv3(x)))
        # x = nn.functional.relu(self.conv4(x))
        x = x.view(x.size()[0],-1)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.relu(self.fc3(x))
        x = self.drop(x)
        x = nn.functional.softmax(self.fc4(x))

        return x

Cost 함수 정의

분류문제에서는 Cross entropy가 주로 loss로 사용된다.

In [0]:
model = Net()
criterion = nn.CrossEntropyLoss()

Optimizer 정의

Optimizer로 Adam 정의해보자. learning Rate는 0.03으로 했을 때 잘 안됐음.
0.001로 하니까 잘 줄어든다.

In [0]:
optimizer = torch.optim.Adam(model.parameters())

In [52]:
MAX_EPOCH = 10
for epoch in range(MAX_EPOCH):
    total_loss = 0
    step = 0
    for X, y in train_loader:
        step+=1
        optimizer.zero_grad()   #gradient 초기화
        output = model(X)
        loss = criterion(output,y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if step%20==0:
            print('{}, {} : {}'.format(epoch,step, total_loss/20))
            total_loss = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0, 20 : 2.2821723341941835
0, 40 : 2.1695340037345887
0, 60 : 2.0499486088752747
0, 80 : 1.9779085040092468
0, 100 : 1.9064203739166259
0, 120 : 1.8806060612201692
0, 140 : 1.8072848856449126
0, 160 : 1.8365394115447997
0, 180 : 1.8225424587726593
0, 200 : 1.8238786220550538
0, 220 : 1.8043528974056244
0, 240 : 1.7959975838661193
0, 260 : 1.810247254371643
0, 280 : 1.8118252456188202
0, 300 : 1.8020312011241912
0, 320 : 1.8152053236961365
0, 340 : 1.7720426738262176
0, 360 : 1.7894958674907684
1, 20 : 1.737954044342041
1, 40 : 1.7671829104423522
1, 60 : 1.7668867826461792
1, 80 : 1.742536151409149
1, 100 : 1.7546024739742279
1, 120 : 1.7664446592330934
1, 140 : 1.7545556366443633
1, 160 : 1.7298399269580842
1, 180 : 1.7389974355697633
1, 200 : 1.7446903467178345
1, 220 : 1.7018438220024108
1, 240 : 1.7497142553329468
1, 260 : 1.716041910648346
1, 280 : 1.707442706823349
1, 300 : 1.7020541727542877
1, 320 : 1.729161411523819
1, 340 : 1.7159496366977691
1, 360 : 1.707181465625763
2, 20 :

In [53]:
total_loss = 0
step = 0
total = 0
correct = 0
with torch.no_grad():
    for val_x, val_y in val_loader:
        step+=1
        val_output = model(val_x)
        v_loss = criterion(val_output, val_y)
        total_loss += v_loss.item()
        _, predicted = torch.max(val_output.data, 1)
        total += val_y.size(0)
        correct += (predicted == val_y).sum().item()
        if step%100 == 0:
            print('{} : {}'.format(step, correct/total))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


100 : 0.794375
200 : 0.78109375
300 : 0.7785416666666667
400 : 0.778828125
500 : 0.7793125
600 : 0.7800520833333333
700 : 0.7810267857142857
800 : 0.7800390625
900 : 0.7790277777777778
1000 : 0.77865625
1100 : 0.7775284090909091
1200 : 0.7777604166666666
1300 : 0.776201923076923
1400 : 0.7764508928571429
1500 : 0.7766666666666666


In [54]:
step = 0
total= 0
correct = 0
total_loss = 0
with torch.no_grad():
    for test_x, test_y in test_loader:
        step += 1
        test_output = model(test_x)
        test_loss = criterion(test_output, test_y)
        total_loss += test_loss.item()
        _, predicted = torch.max(test_output.data, 1)
        total += test_y.size(0)
        correct += (predicted == test_y).sum().item()
        if step%100 == 0:
            print('test data loss : {}, correct : {}'.format(total_loss/100, correct/total))
            total_loss = 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


test data loss : 1.665050367116928, correct : 0.79625
test data loss : 1.6740384519100189, correct : 0.791875
test data loss : 1.692060624361038, correct : 0.7839583333333333
test data loss : 1.684967257976532, correct : 0.78171875
test data loss : 1.6816841506958007, correct : 0.7815625
test data loss : 1.6937005412578583, correct : 0.7792708333333334
test data loss : 1.68610368847847, correct : 0.7787053571428572
test data loss : 1.678886466026306, correct : 0.779140625
test data loss : 1.6910376012325288, correct : 0.7779513888888889
test data loss : 1.6839831733703614, correct : 0.7779375
test data loss : 1.6797351694107057, correct : 0.7782102272727273
test data loss : 1.6872067403793336, correct : 0.7778385416666667
test data loss : 1.6746445643901824, correct : 0.7785096153846154
test data loss : 1.6844160628318787, correct : 0.77828125
test data loss : 1.6948621833324433, correct : 0.7774166666666666
test data loss : 1.6842704784870148, correct : 0.77734375
test data loss : 1.6

conv 사용했을때 : 정확도 72% 정도

fc만 사용했을때 : 정확도 77.8% 정도